In [1]:
import pandas as pd
import numpy as np
import decimal as D

In [2]:
df = pd.read_csv("2021-02-06-SettlementNotice - Visa VSS 910.csv", sep=";", parse_dates=["ReportDate"],  thousands=".", decimal=",", usecols=["ReportDate", "BusinessTransactionCycle", "GrossAmount", "DefermentDays"], dtype={"GrossAmount": np.float64})

In [3]:
df.DefermentDays.unique()

array(['   ', '002', '027'], dtype=object)

In [4]:
df.insert(4, "eventType", "")

In [5]:
for index, row in df.iterrows():
    if row["BusinessTransactionCycle"] == "Originals" and row["DefermentDays"] == "027":
        df.loc[index, "eventType"] = "credit"
    elif row["BusinessTransactionCycle"] == "Originals" and row["DefermentDays"] == "   ":
        df.loc[index, "eventType"] = "debit"

In [6]:
for index, row in df.iterrows():
    if row["BusinessTransactionCycle"] == "Dispute Financial":
        df.loc[index, "eventType"] = "chargeback"
    elif row["BusinessTransactionCycle"] == "Dispute Response Financial":
        df.loc[index, "eventType"] = "secondPresentment"

In [7]:
df["ReportDate"] = pd.to_datetime(df["ReportDate"])

In [8]:
df["BusinessTransactionCycle"] = df["BusinessTransactionCycle"].astype("str")

In [9]:
df.dtypes

ReportDate                  datetime64[ns]
BusinessTransactionCycle            object
GrossAmount                        float64
DefermentDays                       object
eventType                           object
dtype: object

In [10]:
df.head()

,ReportDate,BusinessTransactionCycle,GrossAmount,DefermentDays,eventType
0,2021-02-05,The transaction does not have a cycle.,433.85,,
1,2021-02-05,The transaction does not have a cycle.,5138.17,002,
2,2021-02-05,Dispute Financial,74.79,,chargeback
3,2021-02-05,Originals,71091.71,,debit
4,2021-02-05,Originals,1776864.18,027,credit


In [11]:
df = df.drop(columns=["BusinessTransactionCycle","DefermentDays"])

In [12]:
df

,ReportDate,GrossAmount,eventType
0,2021-02-05,4.338500e+02,
1,2021-02-05,5.138170e+03,
2,2021-02-05,7.479000e+01,chargeback
3,2021-02-05,7.109171e+04,debit
4,2021-02-05,1.776864e+06,credit
5,2021-02-05,5.050209e+05,chargeback
6,2021-02-05,1.686062e+04,secondPresentment
7,2021-02-05,3.204443e+04,
8,2021-02-05,8.404387e+07,debit
9,2021-02-05,1.143416e+08,credit


In [13]:
df = df[['ReportDate', 'eventType', 'GrossAmount']]

In [14]:
df

,ReportDate,eventType,GrossAmount
0,2021-02-05,,4.338500e+02
1,2021-02-05,,5.138170e+03
2,2021-02-05,chargeback,7.479000e+01
3,2021-02-05,debit,7.109171e+04
4,2021-02-05,credit,1.776864e+06
5,2021-02-05,chargeback,5.050209e+05
6,2021-02-05,secondPresentment,1.686062e+04
7,2021-02-05,,3.204443e+04
8,2021-02-05,debit,8.404387e+07
9,2021-02-05,credit,1.143416e+08


In [15]:
df.rename(columns={"ReportDate": "confirmedDate",
                  "GrossAmount": "amountInBRL"}, inplace=True)

In [16]:
#df.drop(df[df['eventType']=""].index, inplace = True) 

In [17]:
rows_to_delete = []

In [18]:
for index, row in df.iterrows():
    if row["eventType"] == "":
        rows_to_delete.append(index)

In [19]:
rows_to_delete

[0, 1, 7, 10, 11, 12, 17]

In [20]:
df = df.drop(rows_to_delete)

In [21]:
dfCamila = df # (Output final)

In [22]:
dfCamila
# ['confirmedDate', 'eventType', 'amountInBRL']) 

,confirmedDate,eventType,amountInBRL
2,2021-02-05,chargeback,7.479000e+01
3,2021-02-05,debit,7.109171e+04
4,2021-02-05,credit,1.776864e+06
5,2021-02-05,chargeback,5.050209e+05
6,2021-02-05,secondPresentment,1.686062e+04
8,2021-02-05,debit,8.404387e+07
9,2021-02-05,credit,1.143416e+08
13,2021-02-06,debit,8.348275e+04
14,2021-02-06,credit,1.832042e+06
15,2021-02-06,chargeback,4.963857e+05


In [23]:
mov = pd.read_csv("MovementByConfirmedDate.csv", sep=';', thousands=",", parse_dates=['ConfirmedDate'], dtype={"GrossAmount": np.float64, "NetAmount": np.float64}, usecols=["ConfirmedDate", "ServiceType", "GrossAmount"])

In [24]:
mov.rename(columns={'ConfirmedDate':'confirmedDate',
                    'ServiceType':'eventType',
                    'GrossAmount':'amountInBRL'},
           inplace=True)

In [25]:
for index, row in mov.iterrows():
    if row["eventType"] in ["Chargeback Suspension", "SecondPresentment Release", "Credit"]:
        mov.loc[index, "eventType"] = "credit"
    elif row["eventType"] == "SecondPresentment":
        mov.loc[index, "eventType"] = "secondPresentment"
    elif row["eventType"] == "Debit":
        mov.loc[index, "eventType"] = "debit"
    elif row["eventType"] == "Chargeback":
        mov.loc[index, "eventType"] = "chargeback"

In [26]:
rows_to_delete = []

In [27]:
for index, row in mov.iterrows():
    if row["eventType"] not in ['chargeback', 'credit', 'debit', 'secondPresentment']:
        rows_to_delete.append(index)

In [28]:
rows_to_delete

[8,
 9,
 42,
 43,
 75,
 76,
 103,
 104,
 114,
 120,
 139,
 140,
 149,
 155,
 166,
 181,
 187,
 192,
 198,
 199,
 224,
 225,
 231,
 236,
 242,
 243,
 268,
 269,
 279,
 280,
 286,
 312,
 318,
 323,
 329,
 330,
 355,
 356,
 362,
 367,
 373,
 374,
 400,
 401,
 411,
 412,
 418,
 439,
 440,
 445,
 449,
 453,
 454,
 464,
 478,
 479,
 485,
 511,
 512,
 518,
 523,
 529,
 550,
 551,
 552,
 562,
 563,
 569,
 593,
 594,
 604,
 610,
 622,
 623,
 636,
 637,
 647,
 653,
 654,
 666,
 667,
 682,
 683,
 693,
 700,
 724,
 725,
 730,
 738,
 739,
 765,
 766,
 771,
 796,
 797,
 807,
 812,
 835,
 836,
 837,
 842,
 847,
 853,
 877,
 878,
 888,
 894,
 905,
 919,
 920,
 926,
 931,
 937,
 962,
 963,
 973,
 979,
 1004,
 1005,
 1010,
 1018,
 1042,
 1043,
 1048]

In [29]:
mov = mov.drop(rows_to_delete)

In [30]:
dfGustavo = mov # (Output final)

In [31]:
vss = dfCamila
receivables = dfGustavo

In [32]:
# Criação tabela de batimento

batimentoCleared = pd.DataFrame(columns = ['confirmedDate', 'eventType', 'vssAmount', 'receivablesAmount', 'dif']) 

###### Check tabelas

In [33]:
vss

,confirmedDate,eventType,amountInBRL
2,2021-02-05,chargeback,7.479000e+01
3,2021-02-05,debit,7.109171e+04
4,2021-02-05,credit,1.776864e+06
5,2021-02-05,chargeback,5.050209e+05
6,2021-02-05,secondPresentment,1.686062e+04
8,2021-02-05,debit,8.404387e+07
9,2021-02-05,credit,1.143416e+08
13,2021-02-06,debit,8.348275e+04
14,2021-02-06,credit,1.832042e+06
15,2021-02-06,chargeback,4.963857e+05


In [34]:
receivables

,confirmedDate,eventType,amountInBRL
0,2021-01-01,credit,-9.584689e+06
1,2021-01-01,credit,2.424347e+07
2,2021-01-01,credit,2.613886e+08
3,2021-01-01,credit,1.000000e+02
4,2021-01-01,credit,4.771980e+05
...,...,...,...
1057,2021-01-24,debit,7.109806e+06
1058,2021-01-24,debit,8.663760e+09
1059,2021-01-24,secondPresentment,5.242800e+04
1060,2021-01-24,secondPresentment,5.322345e+06


In [35]:
batimentoCleared

,confirmedDate,eventType,vssAmount,receivablesAmount,dif


In [36]:
debit = pd.DataFrame(columns = ['confirmedDate', 'amount_Stone', 'amount_Visa', 'dif']) 
credit = pd.DataFrame(columns = ['confirmedDate', 'amount_Stone', 'amount_Visa', 'dif']) 
chargeback = pd.DataFrame(columns = ['confirmedDate', 'amount_Stone', 'amount_Visa', 'dif']) 
secondPresentment = pd.DataFrame(columns = ['confirmedDate', 'amount_Stone', 'amount_Visa', 'dif']) 

###### Chech tabelas de batimento

In [37]:
debit

,confirmedDate,amount_Stone,amount_Visa,dif


In [38]:
credit

,confirmedDate,amount_Stone,amount_Visa,dif


In [39]:
chargeback

,confirmedDate,amount_Stone,amount_Visa,dif


In [40]:
secondPresentment

,confirmedDate,amount_Stone,amount_Visa,dif


In [41]:
# Datas de clearing distintas em cada base
vssDates = vss.confirmedDate.unique()
receivablesDates = receivables.confirmedDate.unique()

# Remoção de duplicatas entre base, composição da base única de datas distintas (allDates)
allDates = np.concatenate((vssDates,receivablesDates))
allUniqueDates = np.unique(allDates)

# Check
allDates

array(['2021-02-05T00:00:00.000000000', '2021-02-06T00:00:00.000000000',
       '2021-01-01T00:00:00.000000000', '2021-02-01T00:00:00.000000000',
       '2021-03-01T00:00:00.000000000', '2021-04-01T00:00:00.000000000',
       '2021-05-01T00:00:00.000000000', '2021-06-01T00:00:00.000000000',
       '2021-07-01T00:00:00.000000000', '2021-08-01T00:00:00.000000000',
       '2021-09-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2021-11-01T00:00:00.000000000', '2021-12-01T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [42]:
## Caso o batimento apresente divergências, uma hipótese a se investigar é o das datas estarem em dois formatos distintos
## Algumas em YYYY-MM-DD e outras em YYYY-DD-MM

In [43]:
# Atribuindo datas distintas ao campo de data dos batimentos
credit.confirmedDate = allDates
debit.confirmedDate = allDates
chargeback.confirmedDate = allDates
secondPresentment.confirmedDate = allDates

In [44]:
# Incluindo amounts das bases originais nas bases de batimento


### Amounts Stone (Receivables)
creditAmounts_Stone = receivables[receivables.eventType == 'credit'].groupby("confirmedDate").sum()

debitAmounts_Stone = receivables[receivables.eventType == 'debit'].groupby("confirmedDate").sum()

chargebackAmounts_Stone = receivables[receivables.eventType == 'chargeback'].groupby("confirmedDate").sum()

secondPresentmentAmounts_Stone = receivables[receivables.eventType == 'secondPresentment'].groupby("confirmedDate").sum()


### Amounts Visa (VSS-910)
creditAmounts_Visa = vss[vss.eventType == 'credit'].groupby("confirmedDate").sum()

debitAmounts_Visa = vss[vss.eventType == 'debit'].groupby("confirmedDate").sum()

chargebackAmounts_Visa = vss[vss.eventType == 'chargeback'].groupby("confirmedDate").sum()

secondPresentmentAmounts_Visa = vss[vss.eventType == 'secondPresentment'].groupby("confirmedDate").sum()

In [45]:
credit

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,NaN,NaN,NaN
1,2021-02-06,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN
6,2021-05-01,NaN,NaN,NaN
7,2021-06-01,NaN,NaN,NaN
8,2021-07-01,NaN,NaN,NaN
9,2021-08-01,NaN,NaN,NaN


In [46]:
# Transformar o confirmedDate dos dataframes parciais de um índice para uma coluna

creditAmounts_Stone.reset_index(inplace=True)
creditAmounts_Stone = creditAmounts_Stone.rename(columns = {'index':'confirmedDate'})

debitAmounts_Stone.reset_index(inplace=True)
debitAmounts_Stone = debitAmounts_Stone.rename(columns = {'index':'confirmedDate'})

chargebackAmounts_Stone.reset_index(inplace=True)
chargebackAmounts_Stone = chargebackAmounts_Stone.rename(columns = {'index':'confirmedDate'})

secondPresentmentAmounts_Stone.reset_index(inplace=True)
secondPresentmentAmounts_Stone = secondPresentmentAmounts_Stone.rename(columns = {'index':'confirmedDate'})

creditAmounts_Visa.reset_index(inplace=True)
creditAmounts_Visa = creditAmounts_Visa.rename(columns = {'index':'confirmedDate'})

debitAmounts_Visa.reset_index(inplace=True)
debitAmounts_Visa = debitAmounts_Visa.rename(columns = {'index':'confirmedDate'})

chargebackAmounts_Visa.reset_index(inplace=True)
chargebackAmounts_Visa = chargebackAmounts_Visa.rename(columns = {'index':'confirmedDate'})

secondPresentmentAmounts_Visa.reset_index(inplace=True)
secondPresentmentAmounts_Visa = secondPresentmentAmounts_Visa.rename(columns = {'index':'confirmedDate'})

Check dataframes

In [47]:
creditAmounts_Stone

,confirmedDate,amountInBRL
0,2021-01-01,4.448963e+09
1,2021-01-13,5.742886e+09
2,2021-01-14,5.146222e+09
3,2021-01-15,7.180038e+09
4,2021-01-16,4.947212e+09
5,2021-01-17,2.956641e+09
6,2021-01-18,1.570865e+10
7,2021-01-19,1.256390e+10
8,2021-01-20,6.148969e+09
9,2021-01-21,6.512597e+09


Testes

In [48]:
for date in allDates:
    print(date)
    print(type(date))
# Com isso, garanto que as datas sendo consideradas dentro do for são as datas corretas

2021-02-05T00:00:00.000000000
<class 'numpy.datetime64'>
2021-02-06T00:00:00.000000000
<class 'numpy.datetime64'>
2021-01-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-02-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-03-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-04-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-05-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-06-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-07-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-08-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-09-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-10-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-11-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-12-01T00:00:00.000000000
<class 'numpy.datetime64'>
2021-01-13T00:00:00.000000000
<class 'numpy.datetime64'>
2021-01-14T00:00:00.000000000
<class 'numpy.datetime64'>
2021-01-15T00:00:00.000000000
<class 'numpy.datetime64'>
2021-01-16T00:00:00.000000000
<

In [49]:
for date in credit.confirmedDate:
    print(type(date))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.tim

In [50]:
primeiraDataDoAllDates = allDates[0]

In [51]:
primeiraDataDaCredit = credit.confirmedDate[0]

In [52]:
primeiraDataDoAllDates

numpy.datetime64('2021-02-05T00:00:00.000000000')

In [53]:
primeiraDataDaCredit

Timestamp('2021-02-05 00:00:00')

In [54]:
primeiraDataDoAllDates == primeiraDataDaCredit

True

In [55]:
#date=allDates[0]
date

Timestamp('2021-01-24 00:00:00')

In [56]:
credit

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,NaN,NaN,NaN
1,2021-02-06,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN
6,2021-05-01,NaN,NaN,NaN
7,2021-06-01,NaN,NaN,NaN
8,2021-07-01,NaN,NaN,NaN
9,2021-08-01,NaN,NaN,NaN


In [ ]:
#Paramos aqui, estamos investigando como conseguir atribuir o valor certo à coluna amount_stone. Vamos tentar a função "merge" do pandas

In [57]:
for date in allDates:
    #Incluindo amounts Stone nas bases de batimento
    
    credit.iat[credit[credit.confirmedDate == date].index.values.astype(int)[0], 1] = creditAmounts_Stone[creditAmounts_Stone.confirmedDate == date].amountInBRL.values.get(0)
    print(creditAmounts_Stone[creditAmounts_Stone.confirmedDate == date].amountInBRL)
    
    #credit[credit.confirmedDate == date].amount_Stone = creditAmounts_Stone[creditAmounts_Stone.confirmedDate == date].amountInBRL
    debit[debit.confirmedDate == date].amount_Stone = debitAmounts_Stone[debitAmounts_Stone.confirmedDate == date].amountInBRL
    chargeback[chargeback.confirmedDate == date].amount_Stone = chargebackAmounts_Stone[chargebackAmounts_Stone.confirmedDate == date].amountInBRL
    secondPresentment[secondPresentment.confirmedDate == date].amount_Stone = secondPresentmentAmounts_Stone[secondPresentmentAmounts_Stone.confirmedDate == date].amountInBRL
    
    #Incluindo amounts Visa nas bases de batimento
    credit[credit.confirmedDate == date].amount_Visa = creditAmounts_Visa[creditAmounts_Visa.confirmedDate == date].amountInBRL
    debit[debit.confirmedDate == date].amount_Visa = debitAmounts_Visa[debitAmounts_Visa.confirmedDate == date].amountInBRL
    chargeback[chargeback.confirmedDate == date].amount_Visa = chargebackAmounts_Visa[chargebackAmounts_Visa.confirmedDate == date].amountInBRL
    secondPresentment[secondPresentment.confirmedDate == date].amount_Visa = secondPresentmentAmounts_Visa[secondPresentmentAmounts_Visa.confirmedDate == date].amountInBRL
    
    
    
    # Fazer batimento em cada data
    credit[credit.confirmedDate == date].dif = credit[credit.confirmedDate == date].amount_Visa - credit[credit.confirmedDate == date].amount_Stone
    debit[debit.confirmedDate == date].dif = debit[debit.confirmedDate == date].amount_Visa - debit[debit.confirmedDate == date].amount_Stone
    chargeback[chargeback.confirmedDate == date].dif = chargeback[chargeback.confirmedDate == date].amount_Visa - chargeback[chargeback.confirmedDate == date].amount_Stone
    secondPresentment[secondPresentment.confirmedDate == date].dif = secondPresentment[secondPresentment.confirmedDate == date].amount_Visa - secondPresentment[secondPresentment.confirmedDate == date].amount_Stone
    

AttributeError: 'numpy.ndarray' object has no attribute 'get'

In [345]:
credit

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,[],NaN,NaN
1,2021-02-06,[],NaN,NaN
2,2021-01-01,[4448962506.0],NaN,NaN
3,2021-02-01,[9338465328.0],NaN,NaN
4,2021-03-01,[10368687977.0],NaN,NaN
5,2021-04-01,[14103846405.0],NaN,NaN
6,2021-05-01,[12200029905.0],NaN,NaN
7,2021-06-01,[12969942778.0],NaN,NaN
8,2021-07-01,[13014076504.0],NaN,NaN
9,2021-08-01,[16079613888.0],NaN,NaN


In [223]:
creditAmounts_Stone

,confirmedDate,amountInBRL
0,2021-01-01,4.448963e+09
1,2021-01-13,5.742886e+09
2,2021-01-14,5.146222e+09
3,2021-01-15,7.180038e+09
4,2021-01-16,4.947212e+09
5,2021-01-17,2.956641e+09
6,2021-01-18,1.570865e+10
7,2021-01-19,1.256390e+10
8,2021-01-20,6.148969e+09
9,2021-01-21,6.512597e+09


In [224]:
## Mais uma hipótese de erro é que tenhamos considerado valores em USD como BRL, porque não tratamos o campo de currency
## das bases originais

Hora da Verdade

In [225]:
debit

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,NaN,NaN,NaN
1,2021-02-06,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN
6,2021-05-01,NaN,NaN,NaN
7,2021-06-01,NaN,NaN,NaN
8,2021-07-01,NaN,NaN,NaN
9,2021-08-01,NaN,NaN,NaN


In [226]:
credit

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,NaN,NaN,NaN
1,2021-02-06,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN
6,2021-05-01,NaN,NaN,NaN
7,2021-06-01,NaN,NaN,NaN
8,2021-07-01,NaN,NaN,NaN
9,2021-08-01,NaN,NaN,NaN


In [227]:
chargeback

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,NaN,NaN,NaN
1,2021-02-06,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN
6,2021-05-01,NaN,NaN,NaN
7,2021-06-01,NaN,NaN,NaN
8,2021-07-01,NaN,NaN,NaN
9,2021-08-01,NaN,NaN,NaN


In [228]:
secondPresentment

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,NaN,NaN,NaN
1,2021-02-06,NaN,NaN,NaN
2,2021-01-01,NaN,NaN,NaN
3,2021-02-01,NaN,NaN,NaN
4,2021-03-01,NaN,NaN,NaN
5,2021-04-01,NaN,NaN,NaN
6,2021-05-01,NaN,NaN,NaN
7,2021-06-01,NaN,NaN,NaN
8,2021-07-01,NaN,NaN,NaN
9,2021-08-01,NaN,NaN,NaN
